# Extract 2D obstacles
- TODO add text
- TODO keep original point cloud and add scalar field

In [1]:
import os
from pathlib import Path
from shapely.geometry import Polygon
import geopandas as gpd

import upcp.utils.bgt_utils as bgt_utils
import upcp.utils.las_utils as las_utils
import upcp.utils.csv_utils as csv_utils

from accessibility import Accessibility # TODO
from las_utils_extra import read_las_m3c2, write_las

## Filter out static points and create a mask
In the previous notebook, we performed a change detection algorithm that calculated M3C2 distance for each point in the point cloud. Based on negative and positive threshold values we can filter for the static points in the point cloud.

In [2]:
# Read point cloud with M3C2 distances
in_file = '../datasets/pointclouds/m3c2/processed_2386_9702.laz'
points, m3c2_distance = read_las_m3c2(in_file)

# Filter for static points
neg_thr = -0.2
pos_thr = 0.2
mask = ((m3c2_distance > neg_thr) & (m3c2_distance < pos_thr))

## Find clusters in the masked point cloud
Using Label Connected Components we find clusters in the point cloud. We only take the Z values of the points to calculate the outer polygon of the clusters.

In [3]:
# 3D Obstacle blobs to 2D polygons using a clustering algorithm and BGT road data.
accesibility_class = Accessibility(min_component_size=100, grid_size=0.05)

static_objects_mask, obstacle_polygons = accesibility_class.get_obstacle_polygons(points, mask)

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [4]:
write_csv_bool = True
write_laz_bool = True

if write_csv_bool:
    csv_headers = ['obstacle']
    bgt_obstacle_file = '../datasets/bgt/bgt_obstacle_demo.csv'

    # Write the csv
    csv_utils.write_csv(bgt_obstacle_file, obstacle_polygons, csv_headers)

if write_laz_bool:
    new_path = '../datasets/pointclouds/obstacles/'
    Path(new_path).mkdir(parents=True, exist_ok=True)
    filename = os.path.basename(in_file)
    write_las(points[static_objects_mask], os.path.join(new_path, filename)) 

Saving LAS lidar data


## Create new sidewalk polygons with obstacles
The found obstacles (polygons) in the previous step are merged with the sidewalk polygons as interiors. 

In [5]:
tilecode = '2386_9702'
((x_min, y_max), (x_max, y_min)) = las_utils.get_bbox_from_tile_code(tilecode, padding=1)

# Create reader for BGT sidewalk part polygons.
bgt_road_file = '../datasets/bgt/bgt_voetpad_demo.csv'
bgt_sidewalk_reader = bgt_utils.BGTPolyReader(bgt_file=bgt_road_file)

sidewalk_polygons = bgt_sidewalk_reader.filter_tile(
                            tilecode, bgt_types=['voetpad'],
                            padding=0, offset=0,
                            merge=False)

In [6]:
# Substract obstacle polygons from sidewalk polygons
sidewalk_with_obstacles = []
for poly_sidewalk in sidewalk_polygons:
    poly_intersect_list = []
    for poly in obstacle_polygons:
        shapely_poly = Polygon(poly[0])
        if poly_sidewalk.contains(shapely_poly): # TODO [0]
            poly_intersect_list.append(shapely_poly)
    
    # Create interiors in sidewalk polygon
    sidewalk_obstacles = poly_sidewalk
    for poly_intersect in poly_intersect_list:
        sidewalk_obstacles -= poly_intersect
        
    sidewalk_with_obstacles.append(sidewalk_obstacles)

In [8]:
# save the new polygons
df = gpd.GeoDataFrame(columns=['geometry'], geometry='geometry') # TODO moet dit?
df['geometry'] = sidewalk_with_obstacles
df.crs = {'init': 'epsg:28992'} #local crs # TODO moet dit?
df.to_file('../datasets/sidewalk_with_obstacles.shp') 

/opt/anaconda3/envs/m3c2/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/opt/anaconda3/envs/m3c2/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
